# Import/Install Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [17]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [18]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [19]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [20]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [21]:
from mediapipe.framework.formats import landmark_pb2


# 3. Extract Keypoint Values

In [22]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [30]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('./MS-ASL_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'nice', 'teacher', 'eat', 'no', 'happy'])

# Thirty videos worth of data
no_sequences = 6

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start useless?
start_folder = 30

In [31]:
for action in actions: 
    # dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(no_sequences):
        try: 
            # os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5(NEW) Process videos for keypoint values

In [32]:
# import json video labels:
import json

json_file_path = 'video_labels.json'

# Load the JSON file into a dictionary
with open(json_file_path, 'r') as file:
    video_labels = json.load(file)

In [33]:
# Mediapipe holistic without manual data collection

def video_to_numpy_array(video_path, holistic, action, sequence_length, sequence_count):
    # Initialize the video capture object with the video file
    cap = cv2.VideoCapture(video_path)

    frames = []  # A list to store each frame of the video as a NumPy array

    # Read frames until the video ends
    i = 0
    while i < sequence_length:
        ret, frame = cap.read()
        if not ret:
            break  # Exit the loop if there are no frames left to read
        frame = cv2.resize(frame, (1280, 720))

        #Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # NEW Export keypoints
        keypoints = extract_keypoints(results)
        npy_path = os.path.join(DATA_PATH, action, str(sequence_count), str(i))
        np.save(npy_path, keypoints)

        i += 1

    # Release the video capture object
    cap.release()        

In [34]:
label_count = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}

# Set mediapipe model 
directory_path = './video-files/'
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for file in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file)
        action = actions[video_labels[file]]
        sequence_count = label_count[video_labels[file]]
        if sequence_count >= no_sequences:
            continue
        # else continue building the frames
        video_to_numpy_array(file_path, holistic, action, sequence_length, sequence_count)

        label_count[video_labels[file]] += 1

I0000 00:00:1711170415.618226       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 76.1), renderer: Apple M1


# 6. Preprocess Data and Create Labels and Features

In [35]:
from sklearn.model_selection import train_test_split
# import tensorflow.keras.utils as tf
from tensorflow.keras.utils import to_categorical

In [36]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'hello': 0, 'nice': 1, 'teacher': 2, 'eat': 3, 'no': 4, 'happy': 5}

In [37]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            print(res)
            window.append(res)
            # if frame_num > 0:
            #     frame_diff = res - prev_frame
            #     window.append(frame_diff)
                
            # Update the previous frame for the next iteration
            prev_frame = res
        sequences.append(window)
        labels.append(label_map[action])

[ 0.44904956  0.3000983  -0.77008748 ...  0.          0.
  0.        ]
[ 0.43930039  0.31763977 -0.59111953 ...  0.          0.
  0.        ]
[ 0.43797803  0.31792989 -0.5262931  ...  0.          0.
  0.        ]
[ 0.43325487  0.31926307 -0.55664933 ...  0.          0.
  0.        ]
[ 0.43119711  0.32387656 -0.52423859 ...  0.          0.
  0.        ]
[ 0.42881003  0.32832482 -0.60879785 ...  0.          0.
  0.        ]
[ 0.4256556   0.32960257 -0.6144765  ...  0.          0.
  0.        ]
[ 0.42362022  0.33048239 -0.63218677 ...  0.          0.
  0.        ]
[ 0.42308491  0.33257928 -0.62973821 ...  0.          0.
  0.        ]
[ 0.4206236   0.33276856 -0.63087267 ...  0.          0.
  0.        ]
[ 0.41987491  0.33318409 -0.62034881 ...  0.          0.
  0.        ]
[ 0.42038769  0.3337909  -0.61418521 ...  0.          0.
  0.        ]
[ 0.42122746  0.33393487 -0.61594903 ...  0.          0.
  0.        ]
[ 0.42185837  0.33408469 -0.61808628 ...  0.          0.
  0.        ]
[ 0.42

In [91]:
np.array(sequences).shape

(36, 30, 1662)

In [39]:
np.array(labels).shape

(36,)

In [92]:
X = np.array(sequences)
X.shape

(36, 30, 1662)

In [93]:
y = to_categorical(labels).astype(int)

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1)


In [95]:
y_test.shape

(1, 6)

# 7. Build and Train LSTM NN

In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [87]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [88]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
# model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(59,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [89]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [90]:
model.fit(X_train, y_train, epochs=750, callbacks=[tb_callback])

Epoch 1/750
2/2 [==============================] - 2s 30ms/step - loss: 1.8060 - categorical_accuracy: 0.1143
Epoch 2/750
2/2 [==============================] - 0s 24ms/step - loss: 1.7812 - categorical_accuracy: 0.2000
Epoch 3/750
2/2 [==============================] - 0s 27ms/step - loss: 2.2077 - categorical_accuracy: 0.2000
Epoch 4/750
2/2 [==============================] - 0s 30ms/step - loss: 1.8305 - categorical_accuracy: 0.1714
Epoch 5/750
2/2 [==============================] - 0s 24ms/step - loss: 1.9845 - categorical_accuracy: 0.1143
Epoch 6/750
2/2 [==============================] - 0s 28ms/step - loss: 1.8401 - categorical_accuracy: 0.1714
Epoch 7/750
2/2 [==============================] - 0s 22ms/step - loss: 1.7892 - categorical_accuracy: 0.1714
Epoch 8/750
2/2 [==============================] - 0s 23ms/step - loss: 1.7871 - categorical_accuracy: 0.1714
Epoch 9/750
2/2 [==============================] - 0s 21ms/step - loss: 1.7961 - categorical_accuracy: 0.1714
Epoch 10/7

In [54]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 6)                 198       
                                                                 
Total params: 596774 (2.28 MB)
Trainable params: 59677

# 8. Make Predictions

In [55]:
res = model.predict(X_test)

1/1 [==============================] - 0s 310ms/step


In [57]:
actions[np.argmax(res[0])]

'hello'

In [58]:
actions[np.argmax(y_test[0])]

'teacher'

# 9. Save Weights

In [ ]:
# commented out to not create a new model
#model.save('new.h5')

In [96]:
del model

In [ ]:
model.load_weights('weights/actionmain.h5')

# 10. Evaluation: Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)